PSBlueSky Demo

Jeff Hick's cool PowerShell Module that uses the BlueSky API and allows you to Skeet from the CLI, and is route to bot making and automation, plus evrything is cooler in the CLI

How to Install

as normal, can grab it from the PowerShell Gallery via Import-module or Install-PSFModule (and for the PSFRamework.nuget users: Install-PSFModule)

In [ ]:
Install-Module -Name PSBluesky

Install-PSResource -Name PSBluesky -Repository PSGallery -TrustRepository

Install-PSFModule -Name PSBlueSky

Jeff wisely recomends you store the API key you need to get from the BlueSky site. 

I have a keepass one setup, can find out more in my blog

Powershell Profiles Keepass And Psbluesky | PowerShell Young Team
https://powershellyoungteam.github.io/2024/11/22/PowerShell-Profiles-KeePass-and-PSBlueSky.html

if you use 2FA you will need to create an App password, there is detailed instructions in the help pdf

Help!

PSBlueSky has some Gucci built in help! check out these Cmdletts as well as Github help and built in Help (get-Help)

In [ ]:
Open-BskyHelp # which will open a PDF version of this document in your default PDF viewer

Get-BskyModuleInfo # which will show you the module commands and has clickable links to online help and module's github page

Start a session

you need to use the creds in your vault!

you can create a session, and if there is a timeout, you can refresh the session


In [ ]:
Start-BskySession -credential $cred

Get-BskySession # confirm that you are connected to the session

Get-BSkySession | Update-BskySession # to update the session



In [ ]:
# Posting! this was my first post..

New-BskyPost -message "It works!! :)" -imagepath C:\users\Wighty\Pictures\BskyPic.PNG -ImageAlt "pic of my first PSBlueSky post" -Verbose

# pass as an object

$param = @{
    Message = "It works!! :)"
    ImagePath = "C:\users\Wighty\Pictures\BskyPic.PNG "
    ImageAlt = "pic of my first PSBlueSky post"
    Verbose = $true
}

New-BskyPost @param

# with a URL, make sure URL is surrounded by Whitespace

New-BskyPost "Cool #PowerShell tip popped up in my terminal today from Daniel Schroeder ( @deadlydog.bsky.social ). In short, watch this Github repo to stay up to date with known PwSh security vulnerabilities and breaking changes:   [PowerShell Annoucements] (https://github.com/PowerShell/Announcements) " -Verbose

# multiline post, save text as string and pass to New-BskyPost

$multiline = @"
I'm really enjoying the #PSBlueSky module. It's a great way to post to Bluesky from #PowerShell. I can't wait to see what other features are added in the future.

#PowerShell
#PSBlueSky
"@

New-BskyPost $multiline -Verbose -WhatIf

In [ ]:
# Feeds and Followers

Get-BskyFeed -Verbose # get your feed (good to filter down by time or limit)

Get-BskyTimeline -Verbose # get your timeline (good to filter down by time or limit)

Get-BskyNotification -Verbose # get your notifications with clickable links to the post

# reposting, need non -default properties URI and CID

Get-BskyTimeline -Limit 1 | Select-Object Author,Date,Text,URI, CID | Publish-BskyPost -Quote "Testing quoting with a PSBluesky command" -whatif

In [ ]:
# User Stuff

Find-BskyUser -UserName "jsnover.com"  # find anyone with PowerShell in their username and Description

Find-BskyUser -UserName "powershell" | Select-Object username # use this to get a list of usernames for demo purposes :)

Get-BskyProfile # Get your profile/bio or pass another users username

Get-BskyFollowers # get your followers

# see how popular you are

$YoungTeamers = Get-BskyFollowers
$followers.count

Write-Output "You have $($YoungTeamers.Count) followers"

Get-BskyFollowing # get who you are following

# Following, Unfollowing and blocking badies!

New-BskyFollow <userhandle>

#or get pipy with it

Get-BskyProfile "PoshYoungTeam" | New-BskyFollow -passthru

Remove-BskyFollow <userhandle>

#or get pipy with it

Get-BskyProfile "PoshYoungTeam" | Remove-BskyFollow -passthru

#blocking

Get-BskyBlockedUser # get who you have blocked

Block-BskyUser -UserName "baduser" -WhatIf

Unblock-BskyUser -UserName "baduser" -WhatIf



6. Resources

• GitHub - PSBlueSky - https://github.com/jdhitsolutions/PSBluesky
Bluesky API Documentation - [HTTP Reference | Bluesky  ](https://docs.bsky.app/docs/category/http-reference)